# <span style="color:#ff5f27"> 👨🏻‍🏫 Custom Transformation Functions</span>

In this tutorial you will learn how to **register custom transformation functions in hopsworks feature store** and use them in **training and inference pipelines**.

## <span style="color:#ff5f27">🗄️ Table of Contents</span>
- [📝 Imports](#1)
- [📝 Creation of Custom Transformation Functions](#2)
    - [🔮 Connecting to Hopsworks Feature Store](#3)
    - [✍🏻 Registering Custom Transformation Functions in Hopsworks](#4)
- [⛳️ Feature Pipeline](#t1)
    - [💽 Loading Data](#t12)
    - [🪄 Creating Feature Groups](#t13)
- [⛳️ Training Pipeline](#t2)
    - [🖍 Feature View Creation](#6)
    - [🧬 Modeling](#7)
    - [💾 Saving the Model in the Model Registry](#8)
- [⛳️ Inference Pipeline](#t3)
    - [📮 Retrieving the Model from the Model Registry](#9)
    - [👨🏻‍⚖️ Batch Prediction](#10)
    - [👨🏻‍⚖️ Real-time Predictions](#11)

<a name='1'></a>
## <span style='color:#ff5f27'> 📝 Imports </span>

In [ ]:
# Importing necessary libraries
import pandas as pd                         # For data manipulation and analysis using DataFrames
import numpy as np                          # For numerical computations and arrays
import os                                   # For operating system-related functions
import joblib                               # For saving and loading model files

import xgboost as xgb                       # For using the XGBoost machine learning library
from sklearn.metrics import accuracy_score  # For evaluating model accuracy

---
<a name='2'></a>
# <span style="color:#ff5f27;">👩🏻‍🔬 Creation of Custom Transformation Functions</span>

Custom transformation function in hopsworks can be defined in Hopsworks using the `udf` decortaor. Transformation functions in hopsworks are defined as pandas udf's. Hopsworks supports the creation of different types of transformations and allows the access of training dataset statistics for features within the transformation function. You can find more information in the documentation [here](https://docs.hopsworks.ai/latest/user_guides/fs/transformation_functions/). 

Below you can see a User-defined function (UDF) `scaler`defined in Hopsworks that accepts one value, along with its statistics and scales the value between 0 and 1 before returning a transformed feature.

In [ ]:
from hopsworks.hsfs.transformation_statistics import TransformationStatistics

@hopsworks.udf(return_type=float, drop=["feature"])
def scaler(feature: pd.Series, statistics=TransformationStatistics("feature")) -> pd.Series:
    return (feature - statistics.feature.min) / (statistics.feature.max - statistics.feature.min)

Let us define one more UDF that can be use to encode features 

In [ ]:
@hopsworks.udf(return_type=int, drop=["feature"])
def label_encoder(feature: pd.Series, statistics=TransformationStatistics("feature")) -> pd.Series:
    unique_data = sorted([value for value in statistics.feature.unique_values])
    value_to_index = {value: index for index, value in enumerate(unique_data)}
    return pd.Series([value_to_index.get(data, -1) if not pd.isna(data) else None for data in feature])

A custom transformation function once defined can be tested by setting its statistics if required, output column names and then providing a pandas series as input.

In [ ]:
from hopsworks.hsfs.statistics import FeatureDescriptiveStatistics

# Assign test statistics since the UDF uses statistics
statistics = [FeatureDescriptiveStatistics(feature_name="feature", min=0, max=10)]
scaler.transformation_statistics = statistics

# Assign output column names 
scaler.output_column_names = ["scaler_feature"]

# Get the excutable UDF based on the transformation statistics
udf = scaler.get_udf()

# Get testing Series
feature = pd.Series([0, 5, 10])

print("⛳️ The Scaled Values are:", udf(feature).values.tolist())

UDF's once defined can be used as [On-Demand transformations](https://docs.hopsworks.ai/latest/user_guides/fs/feature_group/on_demand_transformations/) by attaching the transformation function to a Feature Group or can be used as a [Model-Dependent transformations](https://docs.hopsworks.ai/latest/user_guides/fs/feature_view/model-dependent-transformations/) by attaching the transformaiton function to a feature view. The transformation functions can also be saved in the Hopsworks feature store so that they can be reterived and used later on. 

<a name='3'></a>
## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

The next step is to login to the Hopsworks platform. 

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

<a name='4'></a>
## <span style="color:#ff5f27;"> ✍🏻 Registering Custom Transformation Functions in Hopsworks</span>

The next step is to **register custom transformation functions** in Hopsworks Feature Store.

You can register your transformation function using the `.create_transformation_function()` method with the next parameters:

- `transformation_function` - your custom transformation function.

- `version` - version of your custom transformation function.

Then don't forget to use the `.save()` method in order to persist transformation function in the backend.

In [ ]:
scaler = fs.create_transformation_function(
        scaler, 
        version=1,
    )
scaler.save()

Now let's check if the custom transformation functions is present in the feature store. You can be the function `get_transformation_functions` for this.

In [ ]:
# Check it your transformation functions are present in the feature store
[tf for tf in fs.get_transformation_functions()]

---
<a name='t1'></a>
# <span style="color:#ff5f27;">⛳️ Feature Pipeline </span>

In this section you will load data, create a Hopsworks feature group and insert your dataset into created feature group.

<a name='t12'></a>
## <span style="color:#ff5f27;"> 💽 Loading Data </span>

To begin with, let's load a dataset which contains air quality measurements for different  cities from 2013-01-01 to 2023-04-11.

In [ ]:
# Load the data
df_original = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_pm2_5_eu.csv")
df_original.head(3)

Now let's add a target variable to the DataFrame. For simplicity and for demonstration purposes you will randomly assign either a 0 or a 1 to each row.

In [ ]:
# Generate a binary target column
df_original['target'] = np.random.choice(
    [0, 1], 
    size=len(df_original),
)
df_original.head(3)

You can see that the saved UDF is also present in the feature store along with other built-in transformation functions.

<a name='t13'></a>
## <span style="color:#ff5f27;">🪄 Creating Feature Groups</span>

Now you need to create a Feature Group and insert your dataset.

You will use `.get_or_create_feature_group()` method of the feature store object.

You can read about **Feature Groups** [here](https://docs.hopsworks.ai/3.2/concepts/fs/feature_group/fg_overview/).

In [ ]:
feature_group = fs.get_or_create_feature_group(
    name='feature_group_online',
    description='Online Feature Group',
    version=1,
    primary_key=['city_name', 'date'],
    online_enabled=True,
)    
feature_group.insert(df_original)

---
<a name='t2'></a>
# <span style="color:#ff5f27;">⛳️ Training Pipeline </span>

In the **Training Pipeline** you will register custom transformation functions in the Hopsworks Feature Store, apply them to specific columns in the feature view, split dataset into train and test and train the XGBClassifier. Then you will register your trained model in the Hopsworks Model Registry.

<a name='6'></a>
## <span style="color:#ff5f27;"> 🖍 Feature View Creation</span>

In this part you will use your custom transformation functions from the feature store, build a Query object and create a feature view.

To retrieve your custom transformation function you need to use the `.get_transformation_function()` method by specifying the **name** and **version**.

In [ ]:
# Retrieve scaler transformation function that was saved to the feature store
scaler = fs.get_transformation_function(
    name="scaler",
    version=1,
)

In Hopsworks Feature Store, a Query object allows you to select specific features from a feature group.

`feature_group.select_except(['date'])` selects all columns from the feature group except for the 'date' column.

In [ ]:
# Select features for training data
selected_features = feature_group.select_except(['date'])

# Uncomment this if you would like to view your selected features
# selected_features.show(5)

After creating the Query object, you will create a feature view.

A feature view is a logical representation of data which can be used for real-time serving or batch processing. 

You can read more about **Feature Views** [here](https://docs.hopsworks.ai/3.2/concepts/fs/feature_view/fv_overview/).

In [ ]:
# Get or create a feature view
feature_view = fs.get_or_create_feature_view(
    name='serving_fv',
    version=1,
    query=selected_features,
    # Apply your custom transformation functions to necessary columns
    transformation_functions=[
        label_encoder("city_name"),
        scaler("pm2_5"),
    ],
    labels=['target'],
)

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>
The next step is to create the train-test split of your data.

Let's clarify the next parameters of the `.train_test_split()` method:

- test_size=0.1: This parameter specifies the size of the test set relative to the entire dataset. In this case, the test set will contain 10% of the data, and the train set will have the remaining 90%.

- description='Description of the dataset': A brief description provided for the train-test split dataset, explaining its purpose or any other relevant information.

In [ ]:
# Create a train-test split dataset
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_size=0.1,
    description='Description of the dataset',
)

In [ ]:
X_train.head(3)

In [ ]:
y_train.head(3)

<a name='7'></a>
## <span style="color:#ff5f27;">🧬 Modeling</span>

As a machine learning algorithm you will use the XGBClassifier.

Let's initialize it, fit on train data and then evaluate using Accuracy Score.

In [ ]:
# Initialize XGBClassifier
xgb_classifier = xgb.XGBClassifier()

# Fit the classifier
xgb_classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = xgb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("👮🏻‍♂️ Accuracy:", accuracy)

## <span style="color:#ff5f27;">🗄 Model Registry</span>

In Hopsworks, the Model Registry is a crucial component used to manage and version machine learning models. It acts as a centralized repository where trained models can be stored, tracked, and shared among team members.

By calling `project.get_model_registry()`, the code retrieves a reference to the Model Registry associated with the current Hopsworks project. This reference allows the user to interact with the Model Registry and perform operations such as registering, versioning, and accessing trained machine learning models.
With the Model Registry, data scientists and machine learning engineers can effectively collaborate, track model changes, and easily deploy the best-performing models to production environments.

In [ ]:
mr = project.get_model_registry()

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The next step is to **define input and output schema** of a machine learning model.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train.values)
output_schema = Schema(y_train)
model_schema = ModelSchema(
    input_schema=input_schema,
    output_schema=output_schema,
)

model_schema.to_dict()

<a name='8'></a>
### <span style="color:#ff5f27;">💾 Saving the Model</span>

Now you are ready to register your model in the Hopsworks Moder Registry.

To begin with, let's create the `xgb_model` model directory and save the trained model in this directory.

In [ ]:
model_dir = "xgb_model"

if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save the model
xgb_classifier.save_model(model_dir + "/model.json")

To register your model in the Hopsworks model registry you can use `.create_model()` method with the next parameters:

- name="xgb_model": The name of the model.

- metrics={"Accuracy": accuracy}: The model's performance metrics are specified as a dictionary, with "Accuracy" as the key and the value being the accuracy score computed earlier in the code. This metric represents the accuracy of the model's predictions on the test data.

- description="XGB model": A brief description of the model.

- input_example=X_train.sample(): An example input from the training data (X_train) is used to demonstrate the expected format of the model's input data. It is randomly sampled from X_train.

- model_schema=model_schema: The model schema, which represents the data input and output structure of the model, is specified using the previously defined model_schema.

In [ ]:
# Create a model in the model registry
model = mr.python.create_model(
    name="xgb_model",
    metrics={"Accuracy": accuracy}, 
    description="XGB model",
    input_example=X_train.sample(),
    model_schema=model_schema,
)

model.save(model_dir)

---
<a name='t3'></a>
# <span style="color:#ff5f27;">⛳️ Inference Pipeline </span>

In the **Inference Pipeline** section, you will retrieve your model from Hopsworks Model Registry and utilize this model to make predictions on both Batch Data and Online Feature Vectors.

<a name='9'></a>
## <span style="color:#ff5f27;"> 📮 Retrieving the Model from Model Registry </span>

To retrieve a previously registered machine learning model from the Hopsworks Model Registry you need to use the `.get_model()` method with the next parameters:

- name="xgb_model": The name of the model to be retrieved.

- version=1: The version number of the model to be retrieved.

Then you will download the model from the Model Registry.

In [ ]:
# Retrieve your model from the model registry
retrieved_model = mr.get_model(
    name="xgb_model",
    version=1,
)
saved_model_dir = retrieved_model.download()

In [ ]:
# Initialize the model
model = xgb.XGBClassifier()

# Load the model from a saved JSON file
model.load_model(saved_model_dir + "/model.json")
model

<a name='10'></a>
## <span style="color:#ff5f27;"> 👨🏻‍⚖️ Batch Prediction </span>

Batch prediction is a process in which a trained machine learning model is used to make predictions on a large set of data all at once.

To retrieve batch data from the feature view you need to use `init_batch_scoring` method of the feature view object.

`training_dataset_version` parameter specifies the version number of the training dataset that will be used for scoring.

Then you can use the `.get_batch_data()` method to retrieve batch data.

In [ ]:
# Initialise feature view to retrieve batch data
feature_view.init_batch_scoring(1)

# Retrieve batch data
batch_data = feature_view.get_batch_data()
batch_data.head(3)

Now let's use retrieved model to predict batch data.

In [ ]:
# Predict batch data using retrieved model
predictions_batch = model.predict(batch_data)
predictions_batch[:10]

<a name='11'></a>
## <span style="color:#ff5f27;"> 👨🏻‍⚖️ Real-time Predictions</span>

**Real-time Predictions** is a process of using a trained machine learning model to make predictions on feature vector(s) in real-time. 

To begin with, let's create `to_numpy` function which will transform a feature vector(s) list into a numpy array with a proper shape.

In [ ]:
def to_numpy(feature_vector):
    """
    Converts a given feature vector into a NumPy array with a shape of (-1, 2).

    Parameters:
    vector (list or array-like): The input vector to be converted.

    Returns:
    numpy.ndarray: A NumPy array with a shape of (-1, 2) containing the elements from the input vector.
    """
    return np.array(feature_vector).reshape(-1,2)

The next step is to initialize the feature view for serving and then retrieve a feature vector with specified primary keys.

In [ ]:
# Initialise feature view to retrieve feature vector
feature_view.init_serving(1)

# Retrieve a feature vector
feature_vector = feature_view.get_feature_vector(
    entry = {
        "city_name": 'Amsterdam',
        "date": '2013-01-01',
    }
)
feature_vector

Now you can use your model to predict the feature vector.

In [ ]:
# Predict feature vector using retrieved model
prediction_feature_vector = model.predict(to_numpy(feature_vector))
prediction_feature_vector

In addition, you can retrieve several feature vectors. Just pass primary keys as a list of dictionaries.

In [ ]:
# Retrieve feature vectors from feature store
feature_vectors = feature_view.get_feature_vectors(
    entry = [
        {"city_name": 'Amsterdam', "date": '2013-01-01'},
        {"city_name": 'Amsterdam', "date": '2014-01-01'},
        {"city_name": 'Amsterdam', "date": '2015-01-01'},
        {"city_name": 'Amsterdam', "date": '2016-01-01'},
    ]
)
feature_vectors

Now you can use your model to predict feature vectors.

In [ ]:
# Predict feature vectors using retrieved model
prediction_feature_vectors = model.predict(to_numpy(feature_vectors))
prediction_feature_vectors

---